# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [86]:
# Use this cell to write your code for Task 1
import pandas as pd
df = pd.read_csv('house_sales.csv', parse_dates=True)
missing_city = df['city'].value_counts()[4]
print(missing_city)

73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [87]:
# Use this cell to write your code for Task 2
df['city'].replace('--', 'Unknown', inplace=True)
df['months_listed'] = df['months_listed'].fillna(df['months_listed'].mean()).round(1)
df['house_type'].replace({'Det.':'Detached', 'Semi':'Semi-detached', 'Terr.':'Terraced'}, inplace=True)
df['house_type']= pd.Categorical(df['house_type'], categories=['Terraced', 'Semi-detached', 'Detached'], ordered=True)
df['area'] = df['area'].str.strip(' sq.m').astype('float').round(1)
clean_data = df

# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [88]:
# Use this cell to write your code for Task 3
price_by_rooms = clean_data.groupby('bedrooms')['sale_price'].agg(['mean', 'var']).reset_index().rename(columns={'mean':'avg_price', 'var':'var_price'}).round(1)
print(price_by_rooms)

   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [89]:
# Use this cell to write your code for Task 4
from sklearn.linear_model import LinearRegression
train = pd.read_csv('train.csv', parse_dates=['sale_date'])
validation = pd.read_csv('validation.csv', parse_dates=['sale_date'])
lr = LinearRegression()
lr.fit(train['bedrooms'].values.reshape(-1,1), train['sale_price'])
y_pred = lr.predict(validation['bedrooms'].values.reshape(-1,1))
base_result = pd.DataFrame({'house_id':validation['house_id'], 'price':y_pred})
print(base_result)


     house_id          price
0     1331375  151438.233892
1     1630115  227427.812045
2     1645745  379406.968350
3     1336775  151438.233892
4     1888274  227427.812045
..        ...            ...
295   1986255  379406.968350
296   1896276  379406.968350
297   1758223  227427.812045
298   1752010  151438.233892
299   1651404  379406.968350

[300 rows x 2 columns]


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [90]:
# Use this cell to write your code for Task 5
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
train = pd.get_dummies(data=train, columns=['city']).drop('city_Teasdale', axis=1)
validation = pd.get_dummies(data=validation, columns=['city']).drop('city_Teasdale', axis=1)
train['house_type']= pd.Categorical(train['house_type'], categories=['Terraced', 'Semi-detached', 'Detached'], ordered=True)
train['house_type'] = train['house_type'].cat.codes
validation['house_type']= pd.Categorical(validation['house_type'], categories=['Terraced', 'Semi-detached', 'Detached'], ordered=True)
validation['house_type'] = validation['house_type'].cat.codes
train['sale_date'] = train['sale_date'].apply(lambda x:x.toordinal())
validation['sale_date'] = validation['sale_date'].apply(lambda x:x.toordinal())
train_clean = train.drop('house_id', axis = 1)
validation_clean = validation.drop('house_id', axis = 1)
scaler = StandardScaler()
scaler.fit_transform(train_clean[['sale_date','months_listed', 'bedrooms', 'area']])
scaler.transform(validation_clean[['sale_date','months_listed', 'bedrooms', 'area']])
y_train = train_clean['sale_price']
X_train = train_clean.drop('sale_price', axis = 1)
X_test = validation_clean
ridge = RidgeCV()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
compare_result = pd.DataFrame({'house_id':validation['house_id'], 'price':y_pred})
print(compare_result)
print(ridge.alpha_)

     house_id          price
0     1331375  122991.688999
1     1630115  303857.463149
2     1645745  383942.243985
3     1336775  122795.209791
4     1888274  272124.729471
..        ...            ...
295   1986255  350822.660722
296   1896276  369843.868444
297   1758223  259746.340757
298   1752010  169113.899961
299   1651404  390677.866814

[300 rows x 2 columns]
0.1
